In [ ]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

device = "cuda"
model_id = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, torch_dtype=torch.float16
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

In [ ]:
from transformers.image_utils import load_image

# Define a chat history and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image")
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What color is the frisbee?"},
            {"type": "image"},
        ],
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "The frisbee is"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=False)
prompt = prompt[:-1]  # remove a whitespace too much
prompt


In [5]:
raw_image = load_image("../imgs/frisbee.jpg")

In [ ]:
inputs = processor(images=raw_image, text=prompt, return_tensors="pt").to(
    device, torch.float16
)
inputs.input_ids.shape, inputs.pixel_values.shape

In [ ]:
processor.tokenizer.decode(inputs.input_ids[0])

In [ ]:
import numpy as np

str_toks = processor.tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
n_img_toks = sum(tok == "<image>" for tok in str_toks)
patches_per_side = float(np.sqrt(n_img_toks))
print("num of img tokens", n_img_toks)
print("patches: ", (patches_per_side, patches_per_side))
print("text tokens: ", [tok for tok in str_toks if tok != "<image>"])

In [ ]:
import matplotlib.pyplot as plt

# Convert pixel values to numpy array
image_array = inputs.pixel_values[0].cpu().numpy().transpose(1, 2, 0).astype(np.float32)

# Plot the image
plt.imshow(image_array)
plt.axis("off")  # Turn off axis
plt.show()

In [ ]:
output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
print(processor.decode(output[0], skip_special_tokens=True))